In [1]:
import numpy as np
import pandas as pd
import pickle
import json
import re
from collections import defaultdict

import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import LSTM, Dense, RepeatVector, Masking, TimeDistributed
from tensorflow. keras.utils import plot_model

In [2]:
image_vecs = pd.read_csv(r"Files/image_vectors_with_ids.csv")
image_vecs.head()

,Image ID,Image Vector
0,458752,[4.80218083e-02 9.07960236e-01 1.19339478e+00 ...
1,262146,[7.87118018e-01 2.44311482e-01 6.21284246e-02 ...
2,524291,[1.1351358 0.643366 1.2675045 0.10811964 1...
3,393221,[2.51035309e+00 2.11022353e+00 4.24877226e-01 ...
4,393223,[1.42517245e+00 4.25842971e-01 9.63381290e-01 ...


In [3]:
questionsdf = pickle.load(open(r"Files/bert_small.pkl", "rb"))
questionsdf.head()

,image_id,question,question_id,vec
0,458752,What is this photo taken looking through?,458752000,"(((tf.Tensor(-0.1294253, shape=(), dtype=float..."
1,458752,What position is this man playing?,458752001,"(((tf.Tensor(-0.11989767, shape=(), dtype=floa..."
2,458752,What color is the players shirt?,458752002,"(((tf.Tensor(-0.085942656, shape=(), dtype=flo..."
3,458752,Is this man a professional baseball player?,458752003,"(((tf.Tensor(-0.11925976, shape=(), dtype=floa..."
4,262146,What color is the snow?,262146000,"(((tf.Tensor(-0.04078256, shape=(), dtype=floa..."


In [4]:
question_features = questionsdf['vec'].values

In [5]:
question_features[18].shape

TensorShape([1, 12, 768])

In [6]:
question_features[0].shape

TensorShape([1, 10, 768])

In [7]:
for i in range(question_features.shape[0]):
    a=question_features[i].shape[1]
    question_features[i]=tf.reshape(question_features[i],[1*a*768])

In [8]:
question_features[0].shape

TensorShape([7680])

In [9]:
def concat_tensor(question,image):
    t=tf.concat([question,image], 0)
    s=11600-t.shape[0]
    paddings = tf.constant([[0, s,]])
    t=tf.pad(t, paddings, "CONSTANT")
    return t

In [10]:
lens = []
for quest in questionsdf['question']:
    lens.append(len(quest))
question_length = np.array(lens)
question_length

array([41, 34, 32, ..., 37, 26, 59])

In [11]:
question_length.shape

(9935,)

In [12]:
image_ids = []
for x in questionsdf['image_id']:
    if x not in image_ids:
        image_ids.append(x)
image_ids = np.array(image_ids) - 1
sorted(image_ids)

[8,
 24,
 29,
 33,
 35,
 48,
 60,
 63,
 70,
 71,
 76,
 77,
 80,
 88,
 91,
 93,
 108,
 109,
 112,
 126,
 137,
 141,
 143,
 148,
 153,
 164,
 193,
 200,
 246,
 249,
 259,
 306,
 307,
 308,
 311,
 314,
 320,
 321,
 325,
 331,
 367,
 369,
 381,
 383,
 388,
 393,
 403,
 418,
 430,
 435,
 437,
 442,
 449,
 470,
 507,
 509,
 513,
 528,
 530,
 531,
 539,
 541,
 561,
 571,
 574,
 580,
 583,
 594,
 596,
 604,
 611,
 619,
 624,
 628,
 633,
 642,
 649,
 655,
 658,
 672,
 680,
 689,
 713,
 715,
 721,
 722,
 734,
 753,
 761,
 780,
 789,
 794,
 796,
 824,
 827,
 852,
 881,
 896,
 901,
 907,
 908,
 912,
 924,
 926,
 933,
 940,
 954,
 964,
 981,
 983,
 995,
 1005,
 1013,
 1024,
 1035,
 1058,
 1071,
 1083,
 1089,
 1098,
 1101,
 1106,
 1107,
 1110,
 1121,
 1138,
 1154,
 1165,
 1167,
 1182,
 1199,
 1203,
 1215,
 1223,
 1231,
 1263,
 1270,
 1281,
 1294,
 1305,
 1306,
 1307,
 1310,
 1314,
 1329,
 1331,
 1354,
 1359,
 1365,
 1374,
 1380,
 1385,
 1389,
 1391,
 1396,
 1400,
 1402,
 1406,
 1407,
 1430,
 1452,
 

In [13]:
image_ids.shape

(1913,)

In [16]:
file=r"v2_Annotations_Train_mscoco/v2_mscoco_train2014_annotations.json"
with open(file,'r') as myfile:
    annotations =myfile.read()

In [17]:
annotations=json.loads(annotations)['annotations']

In [18]:
def make_vocab_answers(annotations, n_answers):
    """Make dictionary for top n answers and save them into text file."""
    answers = defaultdict(lambda: 0)
    for annotation in annotations:
            for answer in annotation['answers']:
                word = answer['answer']
                if re.search(r"[^\w\s]", word):
                    continue
                answers[word] += 1
                
    answers = sorted(answers, key=answers.get, reverse=True)
    assert('<unk>' not in answers)
    top_answers = ['<unk>'] + answers[:n_answers-1] # '-1' is due to '<unk>'
    
    with open('Files/vocab_answers.txt', 'w') as f:
        f.writelines([w+'\n' for w in top_answers])

    print('Make vocabulary for answers')
    print('The number of total words of answers: %d' % len(answers))
    print('Keep top %d answers into vocab' % n_answers)

In [19]:
make_vocab_answers(annotations,500) ## this saves a text file in the same folder

Make vocabulary for answers
The number of total words of answers: 135203
Keep top 500 answers into vocab


In [20]:
question_featuresSENTENCE_SPLIT_REGEX = re.compile(r'(\W+)')


def tokenize(sentence):
    tokens = SENTENCE_SPLIT_REGEX.split(sentence.lower())
    tokens = [t.strip() for t in tokens if len(t.strip()) > 0]
    return tokens
def load_str_list(fname):
    with open(fname) as f:
        lines = f.readlines()
    lines = [l.strip() for l in lines]
    return lines


class VocabDict:

    def __init__(self, vocab_file):
        self.word_list = load_str_list(vocab_file)
        self.word2idx_dict = {w:n_w for n_w, w in enumerate(self.word_list)}
        self.vocab_size = len(self.word_list)
        self.unk2idx = self.word2idx_dict['<unk>'] if '<unk>' in self.word2idx_dict else None
    def idx2word(self, n_w):

        return self.word_list[n_w]

    def word2idx(self, w):
        if w in self.word2idx_dict:
            return self.word2idx_dict[w]
        elif self.unk2idx is not None:
            return self.unk2idx
        else:
            raise ValueError('word %s not in dictionary (while dictionary does not contain <unk>)' % w)

    def tokenize_and_index(self, sentence):
        inds = [self.word2idx(w) for w in tokenize(sentence)]

        return inds
        

In [21]:
ans_vocab = VocabDict('Files/vocab_answers.txt')
ans_vocab = ans_vocab.word2idx_dict


In [22]:
answers = np.array(list(ans_vocab.values()))
answers.shape

(500,)

In [23]:
x_tensor=[]
q_id=[]
for i in range(questionsdf.shape[0]):
    image_features = image_vecs[image_vecs['Image ID']==questionsdf.iloc[i]['image_id']]['Image Vector'].apply(lambda x: 
                               np.fromstring(
                               x.replace('\n','')
                                .replace('[','')
                                .replace(']','')
                                .replace('  ',' '), sep=' '))
    image=tf.convert_to_tensor(image_features, np.float32)
    question=question_features[i]
    if(question.shape[0]<11000):
        x_tensor.append(concat_tensor(question,image[0]))
        q_id.append(questionsdf.iloc[i]["question_id"])

In [24]:
def unique_answer(question_id):
    for answer in annotations:
        if answer['question_id']==question_id:
            return answer['multiple_choice_answer']

In [25]:
for i in range(len(x_tensor)):
    x_tensor[i]=x_tensor[i].numpy()

In [26]:
x_train=np.array(x_tensor)
x_train.shape

(9685, 11600)

In [27]:
label=np.zeros(shape=(len(x_tensor),500))
for i in range(len(q_id)):
    ans=unique_answer(q_id[i])
    try:
        key=ans_vocab[ans]
        label[i][key]=1
    except:
        label[i][0]=1

In [28]:
label=np.array(label)
label.shape

(9685, 500)

In [29]:
x_train.shape

(9685, 11600)

In [30]:
h1=200
h2=200
y=500
model = Sequential()
model.add(Dense(h1, input_dim=x_tensor[0].shape[0], activation='relu'))
model.add(Dense(h2, activation='relu'))
model.add(Dense(y, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [31]:
x=x_train
y=label
y.shape

(9685, 500)

In [32]:
history = model.fit(x,y, epochs=50, batch_size=50)

Train on 9685 samples
Epoch 1/50
9685/9685 [==============================] - 19s 2ms/sample - loss: 2.8442 - accuracy: 0.3746
Epoch 2/50
9685/9685 [==============================] - 2s 207us/sample - loss: 2.2680 - accuracy: 0.4165
Epoch 3/50
9685/9685 [==============================] - 2s 210us/sample - loss: 2.0013 - accuracy: 0.4418
Epoch 4/50
9685/9685 [==============================] - 2s 202us/sample - loss: 1.7375 - accuracy: 0.4737
Epoch 5/50
9685/9685 [==============================] - 2s 202us/sample - loss: 1.5048 - accuracy: 0.5161
Epoch 6/50
9685/9685 [==============================] - 2s 217us/sample - loss: 1.2723 - accuracy: 0.5692
Epoch 7/50
9685/9685 [==============================] - 2s 242us/sample - loss: 1.0876 - accuracy: 0.6146
Epoch 8/50
9685/9685 [==============================] - 2s 238us/sample - loss: 0.8888 - accuracy: 0.6816
Epoch 9/50
9685/9685 [==============================] - 2s 240us/sample - loss: 0.7373 - accuracy: 0.7333
Epoch 10/50
9685/9685 [==

In [33]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 200)               2320200   
_________________________________________________________________
dense_1 (Dense)              (None, 200)               40200     
_________________________________________________________________
dense_2 (Dense)              (None, 500)               100500    
Total params: 2,460,900
Trainable params: 2,460,900
Non-trainable params: 0
_________________________________________________________________


In [81]:
def pred_ans(i):
    image_features = image_vecs[image_vecs['Image ID']==questionsdf.iloc[i]['image_id']]['Image Vector'].apply(lambda x: 
                                   np.fromstring(
                                   x.replace('\n','')
                                    .replace('[','')
                                    .replace(']','')
                                    .replace('  ',' '), sep=' '))
    image=tf.convert_to_tensor(image_features, np.float32)
    question=question_features[i]
    t=concat_tensor(question,image[0])
    t=t.numpy()
    t=np.reshape(t,(1,11600))
    p=model.predict_classes(t)
    return p

In [82]:
questionsdf.head()

,image_id,question,question_id,vec
0,458752,What is this photo taken looking through?,458752000,"(tf.Tensor(-0.1294253, shape=(), dtype=float32..."
1,458752,What position is this man playing?,458752001,"(tf.Tensor(-0.11989767, shape=(), dtype=float3..."
2,458752,What color is the players shirt?,458752002,"(tf.Tensor(-0.085942656, shape=(), dtype=float..."
3,458752,Is this man a professional baseball player?,458752003,"(tf.Tensor(-0.11925976, shape=(), dtype=float3..."
4,262146,What color is the snow?,262146000,"(tf.Tensor(-0.04078256, shape=(), dtype=float3..."


In [136]:
pred_id = 99

In [137]:
ans_id=(pred_ans(pred_id))
key_list = list(ans_vocab.keys()) 
val_list = list(ans_vocab.values()) 
  
print(key_list[val_list.index(0)]) 

<unk>


In [138]:
ans =  ans_id[0]

In [139]:
questionsdf.iloc[pred_id]['question']

'How many animals are in this photo?'

In [140]:
ans_vocab['{}'.format(ans)]

6